In [52]:
import pandas as pd
import json

content = json.load(open('data/volume_control/combined_volume.json'))

In [185]:
actions, list_variables, list_num_variables = [], [], []

def entities():
    for i in content.values():
        for key, value in i['entities'].items():
            actions.append(f'{key} = {value}')


def variables():
    variables = ['current_volume', 'volume_max', 'volume_increment', 'volume_decrement', 'last_volume']
    for i in actions:
        count = 0
        var = []
        for j in variables:
            if j in i.split(' = ')[1]:
                count += 1
                var.append(j)
                
        list_variables.append(var) if var!=[] else list_variables.append('None')
        list_num_variables.append(count)


entities()
variables()

df = {
    "Utterance": [i for i in content.keys()],
    "Intent": [i['intent'] for i in content.values()],
    "Action": actions,
    # "Variables": list_variables,
    # "Num_Variables": list_num_variables
}

In [186]:
#! pip3 install openpyxl
data = pd.DataFrame(df)

In [ ]:
# data.to_excel('./data/volume_control/combined_volume.xlsx')

## Bag of words approach

In [88]:
data

,Utterance,Intent,Action,Variables
0,reduce the volume adjuster,volume.decrease,volume = current_volume - volume_decrement,"[current_volume, volume_decrement]"
1,bring down the sound output,volume.decrease,volume = current_volume - volume_decrement,"[current_volume, volume_decrement]"
2,decrease the volume output,volume.decrease,volume = current_volume - volume_decrement,"[current_volume, volume_decrement]"
3,decrease the volume adjuster level,volume.decrease,volume = current_volume - volume_decrement,"[current_volume, volume_decrement]"
4,make the volume setting quieter,volume.decrease,volume = current_volume - volume_decrement,"[current_volume, volume_decrement]"
...,...,...,...,...
6368,turn the volume knob to 31,volume.value,volume = 31,None
6369,adjust the volume to 37 on the system,volume.value,volume = 37,None
6370,increase the volume to 82,volume.value,volume = 82,None
6371,adjust the sound level to 89,volume.value,volume = 89,None


In [83]:
# import tensorflow as tf
# # this is a tokenizer provided by tensorflow
# tokenizer = tf.keras.preprocessing.text.Tokenizer()

# # Create a function to tokenize the text
# def tokenize(text):
#     tokenizer.fit_on_texts(text)
#     return tokenizer.texts_to_sequences(text)


# # data["Utterance"] = tokenize(data["Utterance"])
# # data["Intent"] = tokenize(data["Intent"])
# # data["Action"] = tokenize(data["Action"])

# for i in data.columns:
#     data[i] = tokenize(data[i])

# # Save the vocabulary
# vocab_file = "./data/volume_control/tf_vocab.txt"
# with open(vocab_file, "w") as f:
#     f.write("\n".join(tokenizer.index_word.values()))


In [84]:
data

,Utterance,Intent,Action,Variables
0,"[177, 1, 3, 204]","[1, 13]","[1, 24, 1, 1, 36]","[26, 40]"
1,"[160, 18, 1, 12, 58]","[1, 13]","[1, 24, 1, 1, 36]","[26, 40]"
2,"[33, 1, 3, 58]","[1, 13]","[1, 24, 1, 1, 36]","[26, 40]"
3,"[33, 1, 3, 204, 7]","[1, 13]","[1, 24, 1, 1, 36]","[26, 40]"
4,"[22, 1, 3, 27, 185]","[1, 13]","[1, 24, 1, 1, 36]","[26, 40]"
...,...,...,...,...
6368,"[10, 1, 3, 23, 2, 145]","[1, 3]","[1, 125]",[4]
6369,"[8, 1, 3, 2, 150, 6, 1, 25]","[1, 3]","[1, 130]",[4]
6370,"[31, 1, 3, 2, 141]","[1, 3]","[1, 121]",[4]
6371,"[8, 1, 12, 7, 2, 65]","[1, 3]","[1, 45]",[4]


## Word Embedding approach (Word2Vec)

In [172]:
data

,Utterance,Intent,Action
0,reduce the volume adjuster,volume.decrease,volume = current_volume - volume_decrement
1,bring down the sound output,volume.decrease,volume = current_volume - volume_decrement
2,decrease the volume output,volume.decrease,volume = current_volume - volume_decrement
3,decrease the volume adjuster level,volume.decrease,volume = current_volume - volume_decrement
4,make the volume setting quieter,volume.decrease,volume = current_volume - volume_decrement
...,...,...,...
6368,turn the volume knob to 31,volume.value,volume = 31
6369,adjust the volume to 37 on the system,volume.value,volume = 37
6370,increase the volume to 82,volume.value,volume = 82
6371,adjust the sound level to 89,volume.value,volume = 89


In [195]:
vocab = []

vocab.append(data["Utterance"].astype(str))
vocab.append(data["Intent"].astype(str))
vocab.append(data["Action"].astype(str))

print('Type: ',type(vocab))
print('Length: ',len(vocab))
print(vocab)


Type:  <class 'list'>
Length:  3
[0                   reduce the volume adjuster
1                  bring down the sound output
2                   decrease the volume output
3           decrease the volume adjuster level
4              make the volume setting quieter
                         ...                  
6368                turn the volume knob to 31
6369     adjust the volume to 37 on the system
6370                 increase the volume to 82
6371              adjust the sound level to 89
6372    set the volume to 80 using the speaker
Name: Utterance, Length: 6373, dtype: object, 0       volume.decrease
1       volume.decrease
2       volume.decrease
3       volume.decrease
4       volume.decrease
             ...       
6368       volume.value
6369       volume.value
6370       volume.value
6371       volume.value
6372       volume.value
Name: Intent, Length: 6373, dtype: object, 0       volume = current_volume - volume_decrement
1       volume = current_volume - volume_decr

In [196]:
data

,Utterance,Intent,Action
0,reduce the volume adjuster,volume.decrease,volume = current_volume - volume_decrement
1,bring down the sound output,volume.decrease,volume = current_volume - volume_decrement
2,decrease the volume output,volume.decrease,volume = current_volume - volume_decrement
3,decrease the volume adjuster level,volume.decrease,volume = current_volume - volume_decrement
4,make the volume setting quieter,volume.decrease,volume = current_volume - volume_decrement
...,...,...,...
6368,turn the volume knob to 31,volume.value,volume = 31
6369,adjust the volume to 37 on the system,volume.value,volume = 37
6370,increase the volume to 82,volume.value,volume = 82
6371,adjust the sound level to 89,volume.value,volume = 89


In [208]:
import tensorflow as tf

# create a text vectorizer instance
text_vectorizer = tf.keras.layers.TextVectorization(output_mode='int')

# Vocabulary files
vocab_file_vec = "./data/volume_control/tf_vocab_vec.txt"
# vocab_file_bag = "./data/volume_control/tf_vocab.txt"

# with open(vocab_file_bag, "r") as f:
#     # print(tuple(f.read()))
#     vocab = set()
#     for i in f.read().split('\n'):
#         vocab.add(i)

# text_vectorizer.set_vocabulary(vocabulary=vocab_file_bag)


# Adapt the data to the vectorizer
text_vectorizer.adapt(vocab)
# text_vectorizer.adapt(data["Utterance"].astype(str))
# text_vectorizer.adapt(data["Intent"].astype(str))
# text_vectorizer.adapt(data["Action"].astype(str))


print('Vocab Size: ',text_vectorizer.vocabulary_size())
print('Current Vocab: ',text_vectorizer.get_vocabulary())

# Create a function to tokenize the text
def vectorizer(text):
    # text_vectorizer.adapt(text)
    return text_vectorizer(text)

# loop through the columns and tokenize the text
for i in data.columns:
    data[i] = vectorizer(data[i].astype(str))

# Save the vocabulary
with open(vocab_file_vec, "w") as f:
    f.write("\n".join(text_vectorizer.get_vocabulary()))


Vocab Size:  284
Current Vocab:  ['', '[UNK]', 'volume', 'the', 'volumevalue', 'to', 'set', 'speaker', 'on', 'level', 'adjust', 'of', 'turn', 'audio', 'sound', 'change', 'a', 'scale', 'volumedecrease', 'up', '0', 'down', 'raise', 'lower', 'currentvolume', 'device', 'make', 'knob', '10', 'slider', 'system', 'at', 'volumemax', 'setting', 'volumedecrement', 'increase', 'lastvolume', 'be', 'decrease', 'as', 'amplify', 'control', '100', '99', '98', '97', '96', '95', '94', '93', '92', '91', '90', '9', '89', '88', '87', '86', '85', '84', '83', '82', '81', '80', '8', '79', '78', '77', '76', '75', '74', '73', '72', '71', '70', '7', '69', '68', '67', '66', '65', '64', '63', '62', '61', '60', '6', '59', '58', '57', '56', '55', '54', '53', '52', '51', '50', '5', '49', '48', '47', '46', '45', '44', '43', '42', '41', '40', '4', '39', '38', '37', '36', '35', '34', '33', '32', '31', '30', '3', '29', '28', '27', '26', '25', '24', '23', '22', '21', '20', '2', '19', '18', '17', '16', '15', '14', '13', '1

In [210]:
data

,Utterance,Intent,Action
0,140,1,1
1,140,1,1
2,162,1,1
3,162,1,1
4,1,1,1
...,...,...,...
6368,1,1,1
6369,121,1,1
6370,1,1,1
6371,121,1,1


In [211]:
words = text_vectorizer.get_vocabulary()
print(words)

['', '[UNK]', 'volume', 'the', 'volumevalue', 'to', 'set', 'speaker', 'on', 'level', 'adjust', 'of', 'turn', 'audio', 'sound', 'change', 'a', 'scale', 'volumedecrease', 'up', '0', 'down', 'raise', 'lower', 'currentvolume', 'device', 'make', 'knob', '10', 'slider', 'system', 'at', 'volumemax', 'setting', 'volumedecrement', 'increase', 'lastvolume', 'be', 'decrease', 'as', 'amplify', 'control', '100', '99', '98', '97', '96', '95', '94', '93', '92', '91', '90', '9', '89', '88', '87', '86', '85', '84', '83', '82', '81', '80', '8', '79', '78', '77', '76', '75', '74', '73', '72', '71', '70', '7', '69', '68', '67', '66', '65', '64', '63', '62', '61', '60', '6', '59', '58', '57', '56', '55', '54', '53', '52', '51', '50', '5', '49', '48', '47', '46', '45', '44', '43', '42', '41', '40', '4', '39', '38', '37', '36', '35', '34', '33', '32', '31', '30', '3', '29', '28', '27', '26', '25', '24', '23', '22', '21', '20', '2', '19', '18', '17', '16', '15', '14', '13', '12', '11', '1', 'minimize', 'dimin